# PLL plates

In [ ]:
library(ggplot2)
library(extrafont)
library(stringr)
library(heatmaply)
library(ggrepel)
library(reticulate)
library(gridExtra)
library(robustbase)
library(ggpubr)

In [ ]:
# Load external Python functions
source_python("reticulate_functions.py")
# Load extra fonts
loadfonts()
# Set this to true for faster compilation or false for more precise results and all outputs
TEST_MODE = T

In [ ]:
# Number of permutations used for empirical p-value computations
PERM_NB_ITER = ifelse(TEST_MODE, 20, 2000)

In [ ]:
# Customize ggplot appearance

# Change theme
customTheme <- theme_light() + 
               theme(panel.grid.minor=element_blank(), text=element_text(size=17, family="Arial", colour = "#333333"),
                     line=element_line(colour = "#333333"), legend.background = element_rect(fill=alpha('#CCCCCC', 0.1)), legend.key = element_blank())

# Change default colors
scale_colour_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_colour_gradient(...), 
        viridis = scale_colour_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))
}
scale_color_continuous <- scale_colour_continuous

scale_fill_continuous <- function (..., begin = 0.1, end = 0.9, direction = -1, option = "plasma", 
                                     type = getOption("ggplot2.continuous.colour", default = "viridis")) {
    switch(type, gradient = scale_fill_gradient(...), 
        viridis = scale_fill_viridis_c(option = option, begin = begin, end = end, direction = direction, ...), 
        stop("Unknown scale type", call. = FALSE))

}

cemm_pal = colorRampPalette(c("#5A463C", "#008CAD", "#40B9D4", "#D4ECF2", "#D2323C", "#F8B100", "#DFDC00"))
cust_pal = colorRampPalette(c("#008CAD", "#40B9D4", "#D4ECF2", "#F8B100", "#C00000", "#2D0000"))
yolla_pal = colorRampPalette(c('#FD0F91', '#C00000', '#2D0000'))
scale_fill_discrete <- function (..., type = "Cust", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "fill") 
{
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else if (type == "Yolla"){
        discrete_scale(aesthetics, "Yolla", yolla_pal, na.value = na.value, ...)
    } else if (type == "Cust"){
        discrete_scale(aesthetics, "Cust", cust_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}

scale_color_discrete <- function (..., type = "Cust", h = c(0, 360) + 15, c = 100, l = 65, h.start = 0, 
    direction = 1, na.value = "grey50", aesthetics = "colour") {
    if (type == "CeMM"){
        discrete_scale(aesthetics, "CeMM", cemm_pal, na.value = na.value, ...)
    } else if (type == "Cust"){
        discrete_scale(aesthetics, "Cust", cust_pal, na.value = na.value, ...)
    } else if (type == "Yolla"){
        discrete_scale(aesthetics, "Yolla", yolla_pal, na.value = na.value, ...)
    } else {
        discrete_scale(aesthetics, "hue", scales::hue_pal(h, c, l, h.start, 
            direction), na.value = na.value, ...)
    }
}
scale_colour_discrete <- scale_color_discrete

noGridTheme <- function(...){
    theme(panel.grid.major=element_blank(), axis.text.x=element_text(size=12), axis.text.y=element_text(size=12),
                      axis.line=element_line(color="#333333", size = 0.2), panel.border = element_blank(), ...)
}

darkTheme <- function(...){
    theme(panel.background = element_rect(fill = '#333333'), plot.background = element_rect(fill = '#333333'), 
          axis.line=element_line(color="#CCCCCC", size = 0.2), 
          text=element_text(size=17, family="Arial", colour = "#CCCCCC"),
          line=element_line(colour = "#CCCCCC"))
}

theme_set(customTheme)

options(repr.plot.width=10, repr.plot.height=10)

## Jurkat - PLL and stimulatory coatings - Intensity per cell

In [ ]:
# Fix t-SNE layout and random number generation
set.seed(38)

In [ ]:
FILT_MIN_CELLS = 2
FILT_MAX_INT_DNA = 0.05 # Remove empty images and small DNA precipitations

In [ ]:
Cells = read.csv("Rsc/allCytoplasms_JK_WT_activation.csv", header=T)
Images = read.csv("Rsc/allImages_JK_WT_activation.csv", header=T)

In [ ]:
Images$Plate = str_extract(as.character(Images$URL_Actin), "Jurkat_P[^/]*") 

In [ ]:
getPlate <- function(image){
    Images$Plate[Images$ImageNumber == image]
}
Cells$Plate <- sapply(Cells$ImageNumber, getPlate)

In [ ]:
Cells$Plate = as.factor(Cells$Plate)
levels(Cells$Plate) = c("ICAM-1 plate 1", "ICAM-1 plate 2", "ICAM-1 plate 3", "PLL")

In [ ]:
# Remove flagged images with no count information
fieldToKeep <- which(!is.na(Images$Count_FilteredNucleus))
# Remove wells with low max DNA intensity
fieldToKeep <- fieldToKeep[Images$ImageQuality_MaxIntensity_DNA[fieldToKeep] >= FILT_MAX_INT_DNA]
# Remove wells with low cell count
fieldToKeep <- fieldToKeep[Images[fieldToKeep,]$Count_FilteredNucleus >= FILT_MIN_CELLS]

In [ ]:
cellsToKeep = Cells$ImageNumber %in% Images$ImageNumber[fieldToKeep]

In [ ]:
gp <- ggplot(Cells[cellsToKeep,]) + geom_density(aes(x = Intensity_MeanIntensity_CorrActin, color = Plate), 
                                           size = 1.5, adjust = 1) + theme(legend.position = "bottom") +
    scale_color_discrete(type = "Yolla") + ylab("Density") + xlab("Mean actin intensity") +
    guides(color = guide_legend(nrow=2,byrow=TRUE))
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_intensityPerCell.pdf", plot = gp)
}

## Jurkat - PLL

In [ ]:
# Fix t-SNE layout and random number generation
set.seed(38)

In [ ]:
JK = Images
JK$Plate <- str_remove(string = JK$Plate, pattern = "Jurkat_")

## Visualize full dataset

### Cell count

In [ ]:
gpJK = ggplot(JK[!is.na(JK$Count_FilteredNucleus),]) + geom_histogram(aes(Count_FilteredNucleus, 
                                                                         fill = as.factor(Plate)), binwidth=2) +
       scale_fill_discrete(name="Plate")
gpJK

if(!TEST_MODE){
    ggsave(filename = "Fig/JK_WT_activation_count.pdf", plot = gpJK)
}

In [ ]:
gpJK = ggplot(JK[!is.na(JK$Mean_FilterCytoplasm_AreaShape_Area),]) + 
       geom_histogram(aes(Mean_FilterCytoplasm_AreaShape_Area, fill = as.factor(Plate)), binwidth=100) +
       scale_fill_discrete(name="Plate")
gpJK
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_WT_activation_area.pdf", plot = gpJK)
}

In [ ]:
ggplot(JK) + geom_point(aes(Count_Nucleus, Count_FilteredNucleus, color = as.factor(Plate)), 
                        position = "jitter") +
             scale_color_discrete(name="Plate")

## Filtering

In [ ]:
FILT_MAX_INT_DNA = 0.05 # Remove empty images and small DNA precipitations
FILT_MIN_CELLS = 2
FILT_NB_MAX_NA_IMAGE = 353 # Most images generate up to 352 missing values (no LFA-1 granules)
FILT_MAX_CORR = 0.6 # Keep uncorrelated variables
dimUMAP = 3

In [ ]:
ftToKeep = 1:dim(JK)[2]
# Make sure that the fields are numeric
ftToKeep <- ftToKeep[which(sapply(JK[,ftToKeep], class) == "numeric")]
# Remove execution time and count features
ftToKeep <- ftToKeep[grep("(Execution)|(Count)", colnames(JK)[ftToKeep], invert = T)]

In [ ]:
# Store all remaining features before filtering for downstream comparison to selected set
preFiltFt = colnames(JK)[ftToKeep]

In [ ]:
# Remove wells with low max DNA intensity
fieldToKeep <- which(JK$ImageQuality_MaxIntensity_DNA >= FILT_MAX_INT_DNA)
# Remove wells with low cell count
fieldToKeep <- fieldToKeep[JK[fieldToKeep,]$Count_FilteredNucleus >= FILT_MIN_CELLS]

In [ ]:
# Few bad quality pictures are generating a lot of missing values and are removed
fieldToKeep <- fieldToKeep[rowSums(is.na(JK[fieldToKeep,ftToKeep])) < FILT_NB_MAX_NA_IMAGE]
# Remove remaining features with missing values
ftToKeep <- ftToKeep[colSums(is.na(JK[fieldToKeep,ftToKeep])) == 0] 
# Remove constant columns
ftToKeep <- ftToKeep[sapply(JK[fieldToKeep,ftToKeep], function(x) mad(x) != 0)]

In [ ]:
# Approximate normal distribution
transfLog <- function (x){
    minVal = min(x)
    return(log(x+1-minVal))
}
transformedJK = as.data.frame(apply(JK[fieldToKeep, ftToKeep], 2, transfLog))

In [ ]:
# Center and scale on all images from same plate
transfNorm <- function(x, y){
    return((x - median(y))/mad(y))
}
for (plate in levels(JK$Plate)){
    indPlateField = JK$Plate[fieldToKeep] == plate
    
    transformedJK[indPlateField,] = apply(
        transformedJK[indPlateField,], 2, 
        function(x) transfNorm(x, x))
}

In [ ]:
# Remove columns with NA, i.e.
# features with mad == 0 for 1 plate or more
noNAFt = colSums(is.na(transformedJK)) == 0
ftToKeep = ftToKeep[noNAFt]
transformedJK = transformedJK[,noNAFt]

In [ ]:
# Order features from biggest mad to smallest mad
# Since features have mad(WT) = 1 on each plate, it means that we rank features by how more variable they are
# for drug perturbations than for WT
orderFt = rev(order(apply(transformedJK, 2, mad)))

In [ ]:
uncorrFt = uncorrelate(transformedJK, orderCol = orderFt-1, threshold = FILT_MAX_CORR)
uncorrFt = unlist(uncorrFt) + 1

In [ ]:
transformedJK = transformedJK[,uncorrFt]

In [ ]:
dim(JK)
dim(transformedJK)

### Export subset of features

For Jurkat cells, the following features are selected and explored separately for their biological interpretability:
```
* Actin intensity/cell (mean/well): JK$Intensity_MeanIntensity_CorrActin_FilterCytoplasm
* Cell area: JK$Mean_FilterCytoplasm_AreaShape_Area
* Cell roundness: JK$Mean_FilterCytoplasm_AreaShape_FormFactor
* Cell width: JK$Mean_FilterCytoplasm_AreaShape_MajorAxisLength 
* Cell length: JK$Mean_FilterCytoplasm_AreaShape_MinorAxisLength
* Cell length to width ratio: JK$Mean_FilterCytoplasm_AreaShape_MinorAxisLength / JK$Mean_FilterCytoplasm_AreaShape_MajorAxisLength
* Average number of LFA1 granules / cell: JK$Count_PerfGranules  / JK$Count_FilterCytoplasm
* LFA1 area / cell area: (JK$Count_PerfGranules * JK$Mean_PerfGranules_AreaShape_Area)  / (JK$Count_FilterCytoplasm * JK$Mean_FilterCytoplasm_AreaShape_Area)
* LFA1 intensity: JK$Intensity_MeanIntensity_CorrPerf_FilterCytoplasm
* LFA1 area: JK$Mean_PerfGranules_AreaShape_Area
* Nucleus intensity: JK$Intensity_MeanIntensity_CorrDNA_FilteredNucleus
* Nucleus area: JK$Mean_FilteredNucleus_AreaShape_Area
* Nucleus roundness: JK$Mean_FilteredNucleus_AreaShape_FormFactor
* Nucleus width: JK$Mean_FilteredNucleus_AreaShape_MajorAxisLength 
* Nucleus length: JK$Mean_FilteredNucleus_AreaShape_MinorAxisLength
* Nucleus ratio: JK$Mean_FilteredNucleus_AreaShape_MinorAxisLength / JK$Mean_FilteredNucleus_AreaShape_MajorAxisLength
* Nucleus area / cell area: (JK$Count_FilteredNucleus * JK$Mean_FilteredNucleus_AreaShape_Area)  / (JK$Count_FilterCytoplasm * JK$Mean_FilterCytoplasm_AreaShape_Area)```

NB (from CellProfiler docs): FormFactor = $4 \times π \times Area / Perimeter^2$. Equals 1 for a perfectly circular object.

In [ ]:
subsetJK = cbind(Field = str_extract(as.character(JK[fieldToKeep,]$URL_Actin), "r..c..f.."),
                 Plate = JK[fieldToKeep,]$Plate,
                 ActinIntensity = JK[fieldToKeep,]$Intensity_MeanIntensity_CorrActin_FilterCytoplasm,
                 CellArea = JK[fieldToKeep,]$Mean_FilterCytoplasm_AreaShape_Area,
                 CellRoundness = JK[fieldToKeep,]$Mean_FilterCytoplasm_AreaShape_FormFactor,
                 CellWidth = JK[fieldToKeep,]$Mean_FilterCytoplasm_AreaShape_MajorAxisLength,
                 CellLength = JK[fieldToKeep,]$Mean_FilterCytoplasm_AreaShape_MinorAxisLength,
                 CellLengthOverWidth = JK[fieldToKeep,]$Mean_FilterCytoplasm_AreaShape_MinorAxisLength / 
                     JK[fieldToKeep,]$Mean_FilterCytoplasm_AreaShape_MajorAxisLength,
                 LfaIntensity = JK[fieldToKeep,]$Intensity_MeanIntensity_CorrLFA_FilteredLFA,
                 LfaActinCorrelation = JK[fieldToKeep,]$Mean_FilterCytoplasm_Correlation_Correlation_CorrLFA_CorrActin,
                 LfaArea = JK[fieldToKeep,]$Mean_FilteredLFA_AreaShape_Area,
                 LfaAreaOverCellArea = JK[fieldToKeep,]$Mean_FilteredLFA_AreaShape_Area / 
                     JK[fieldToKeep,]$Mean_FilterCytoplasm_AreaShape_Area,
                 NucleusIntensity = JK[fieldToKeep,]$Intensity_MeanIntensity_CorrDNA_FilteredNucleus,
                 NucleusArea = JK[fieldToKeep,]$Mean_FilteredNucleus_AreaShape_Area,
                 NucleusRoundness = JK[fieldToKeep,]$Mean_FilteredNucleus_AreaShape_FormFactor,
                 NucleusWidth = JK[fieldToKeep,]$Mean_FilteredNucleus_AreaShape_MajorAxisLength,
                 NucleusLength = JK[fieldToKeep,]$Mean_FilteredNucleus_AreaShape_MinorAxisLength,
                 NucleusLengthOverWidth = JK[fieldToKeep,]$Mean_FilteredNucleus_AreaShape_MinorAxisLength / 
                     JK[fieldToKeep,]$Mean_FilteredNucleus_AreaShape_MajorAxisLength,
                 NucleusAreaOverCellArea = (JK[fieldToKeep,]$Count_FilteredNucleus * JK[fieldToKeep,]$Mean_FilteredNucleus_AreaShape_Area) / 
                 (JK[fieldToKeep,]$Count_FilterCytoplasm * JK[fieldToKeep,]$Mean_FilterCytoplasm_AreaShape_Area)
                )

In [ ]:
# Export list of "interpretable" features
if(!TEST_MODE){
    write.csv(subsetJK, "Tab/JK_WT_activation_features.csv", row.names = F)
}

In [ ]:
CountPerPlate = aggregate(JK[fieldToKeep,]$Count_FilterCytoplasm, by = list(JK[fieldToKeep,]$Plate), 
                          FUN = function(x) sum(x, na.rm = T))
names(CountPerPlate) <- c("Plate", "Count")
# Reminder: Only a single shRNA per gene on the Jurkat PLL plate

In [ ]:
# Export cell counts
if(!TEST_MODE){
    write.csv(CountPerPlate, "Tab/JK_WT_activation_count_plate.csv", row.names = F)
}

### Look at which types of features are kept

In [ ]:
catChannel = c("CorrDNA", "CorrActin", "CorrLFA")
table(rowSums(sapply(catChannel, function(x) grepl(x, colnames(transformedJK)))))
dtCat = data.frame(CountIni = sort(colSums(sapply(catChannel, function(x) grepl(x, preFiltFt)))), 
                   Count = sort(colSums(sapply(catChannel, function(x) grepl(x, colnames(transformedJK))))))

dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) + ylim(c(0,750)) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 2, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_WT_activation_SelecFt_Channel.pdf", plot = gp, width = 10)
}

In [ ]:
catObjects = c("ActinGranules", "FilterCytoplasm", "ShrunkenCytoplasm", "FilteredNucleus", "FilteredLFA")
table(rowSums(sapply(catObjects, function(x) grepl(x, colnames(transformedJK)))))
dtCat = data.frame(CountIni = colSums(sapply(catObjects, function(x) grepl(x, preFiltFt))), 
                   Count = colSums(sapply(catObjects, function(x) grepl(x, colnames(transformedJK)))))

dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) + ylim(c(0,600)) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 2, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_WT_activation_SelecFt_Object.pdf", plot = gp, width = 10)
}

In [ ]:
catType = c("Threshold", "Granularity", "ImageQuality", "Texture", "AreaShape", "RadialDistribution", "Neighbors", 
            "Correlation", "Intensity", "Overlap", "Location")
which(rowSums(sapply(catType, function(x) grepl(x, colnames(transformedJK)))) == 0) # All features are covered

dtCat = data.frame(CountIni = colSums(sapply(catType, function(x) grepl(x, preFiltFt))), 
                   Count = colSums(sapply(catType, function(x) grepl(x, colnames(transformedJK)))))
dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 5, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_WT_activation_SelecFt_Type_All.pdf", plot = gp, width = 10)
}

In [ ]:
catType = c("Granularity", "Texture", "AreaShape", "RadialDistribution",
            "Correlation", "Intensity")
table(rowSums(sapply(catType, function(x) grepl(x, colnames(transformedJK)))))

dtCount = as.data.frame(sapply(catType, function(x) grepl(x, colnames(transformedJK))))
dtCount$Other = !apply(dtCount, 1, any)
dtCountIni = as.data.frame(sapply(catType, function(x) grepl(x, preFiltFt)))
dtCountIni$Other = !apply(dtCountIni, 1, any)
dtCat = data.frame(CountIni = colSums(dtCountIni), 
                   Count = colSums(dtCount))
dtCat$Order <- rank(dtCat$CountIni, ties.method = c("first"))
dtCat$Category <- factor(rownames(dtCat), levels=rownames(dtCat)[order(dtCat$CountIni)])
dtCat$Ratio <- dtCat$Count / dtCat$CountIni
dtCat$Ratio <- paste0(round(100*dtCat$Ratio, 1), "%")
gp <- ggplot(dtCat) + geom_bar(aes(Category, weight = CountIni), fill = cust_pal(2)[1]) +
              geom_bar(aes(Category, weight = Count), fill = cust_pal(2)[2]) + 
              geom_text(aes(x = Order, y = CountIni + 5, label = Ratio), hjust = 0) + coord_flip()
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_WT_activation_SelecFt_Type_Short.pdf", plot = gp, width = 10)
}

In [ ]:
# Export list of features kept
if(!TEST_MODE){
    write.table(colnames(transformedJK), file = "Tab/JK_WT_activation_list_features.csv", col.names = F, row.names = F)
}

### Look at the morphological distribution of the fields of view

In [ ]:
# Fix random number generation
set.seed(38)

In [ ]:
umTJK = umap(transformedJK, min_dist = 0.1, neighbors = 10, n = dimUMAP, metric = "euclidean")
umTJK = as.data.frame(umTJK)
names(umTJK) = paste0("UMAP", 1:dimUMAP)

In [ ]:
umTJK$Row <- as.factor(JK$Metadata_Row[fieldToKeep])
umTJK$Col <- as.factor(JK$Metadata_Column[fieldToKeep])
umTJK$URL <- as.factor(JK$URL_Actin[fieldToKeep])
umTJK$Plate <- as.factor(JK$Plate[fieldToKeep])

In [ ]:
gp <- ggplot(umTJK) + geom_point(aes(UMAP1, UMAP2, color = Plate))
gp

In [ ]:
gp <- ggplot(umTJK) + geom_point(aes(UMAP1, UMAP2, color = Plate)) + xlim(215,221) + ylim(212,217)
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_WT_activation_UMAP_Plates_zoom_center.pdf", plot = gp, width = 10)
}

#### UMAP of morphological features

In [ ]:
umJKFT = umap(t(transformedJK), min_dist = 0.1, neighbors = 10, n = 2, metric = "euclidean")
umJKFT = as.data.frame(umJKFT)
names(umJKFT) = c("UMAP1", "UMAP2")

In [ ]:
catType = c("Granularity", "Texture", "AreaShape", "RadialDistribution",
            "Correlation", "Intensity")
umJKFT$type = apply(as.data.frame(sapply(catType, function(x) 
    grepl(x, colnames(transformedJK)))), 1, function(x) paste(catType[which(x)],collapse='-'))
umJKFT$type[umJKFT$type == ""] <- NA

In [ ]:
catChannel = c("CorrDNA", "CorrActin", "CorrPerf")
umJKFT$channel = apply(as.data.frame(sapply(catChannel, function(x) 
    grepl(x, colnames(transformedJK)))), 1, function(x) paste(catChannel[which(x)],collapse='-'))
umJKFT$channel[umJKFT$channel == ""] <- NA

In [ ]:
gp <- ggplot(umJKFT) + geom_point(aes(UMAP1, UMAP2, col = type, shape = channel), size = 3)
gp

In [ ]:
if(!TEST_MODE){
    ggsave(filename = "Fig/JK_WT_activation_UMAP_Features.pdf", gp, width = 10)
}

In [ ]:
sessionInfo()